# Experiencing real data

In [1]:
import glob
import os
import numpy as np
import math
import pandas as pd
from IPython.display import display
import pickle
from sklearn.externals import joblib

from extract_feature_IMU import extract_feature_IMU

# Preload

In [2]:
#將Pred結果展開成sample比較 #Overlapping 實際執行秒數 = (Sec*Num)-[(Num-1)*Overlap]
def pred_flatToSamples_IMU(y_pred):
    allofSec = windowSize_sec*len(y_pred)-(len(y_pred)-1)*overlapping
    y_pred_sample = [0] * (allofSec*sampleRate_IMU_acc)
    for pred, index in zip(y_pred, range(0, allofSec, windowSize_sec-overlapping)):
        if pred == 1:
            start = index*sampleRate_IMU_acc
            end = start + sampleRate_IMU_acc*windowSize_sec
            listofOne = [1] * (sampleRate_IMU_acc*windowSize_sec)
            y_pred_sample[start:end] = listofOne
    return y_pred_sample

In [3]:
tree_IMU = joblib.load('./models/tree_IMU.pkl')
forest_IMU = joblib.load('./models/forest_IMU.pkl')
knn_IMU = joblib.load('./models/knn_IMU.pkl')
svm_linear_IMU = joblib.load('./models/svm_linear_IMU.pkl')
svm_poly_IMU = joblib.load('./models/svm_poly_IMU.pkl')
svm_sigmoid_IMU = joblib.load('./models/svm_sigmoid_IMU.pkl')
NN_IMU = joblib.load('./models/NN_IMU.pkl')

In [4]:
def ensemble(X_train):
    y_pred_ensemble = []
    y_pred_ensemble.append(tree_IMU.predict(X_train))
    y_pred_ensemble.append(forest_IMU.predict(X_train))
    y_pred_ensemble.append(knn_IMU.predict(X_train))
    y_pred_ensemble.append(svm_linear_IMU.predict(X_train))
    y_pred_ensemble.append(svm_poly_IMU.predict(X_train))
    y_pred_ensemble.append(svm_sigmoid_IMU.predict(X_train))
    y_pred_ensemble.append(NN_IMU.predict(X_train))
    y_pred_ensemble = np.array(y_pred_ensemble).T
    y_pred = []
    for y_seg in y_pred_ensemble:
        y_seg = y_seg.tolist()
        if y_seg.count(0) > y_seg.count(1):
            y_pred.append(0)
        else:
            y_pred.append(1)
    #     unique, counts = numpy.unique(y_seg, return_counts=True)
    #     dict(zip(unique, counts))
    return np.array(y_pred)

# Main

In [5]:
#Loading data
IMU_rw_file_path = r'C:/Users/NCTU/Desktop/WhoTakeWhat/data/IMU/forRO/testing'                     # use your path
IMU_rw_all_files = glob.glob(os.path.join(IMU_rw_file_path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
#Loading configuration
config_path = './config/'
config = pickle.load(open(config_path + 'config.pkl', 'rb'))
sampleRate_IMU_acc = config['sampling_rate']
windowSize_sec = config['windows_size']

for file in IMU_rw_all_files:
    who = file.split('\\')[-1].split('.')[0]
    IMU_rW_pd = pd.read_csv(file, header=None)
    IMU_rW_pd.columns = ['earx', 'eary', 'earz', 'accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz', 'timestamp']

    IMU_rW = IMU_rW_pd[['accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz']].values.tolist()

    ##Segmentation(by windows Size, and Overlapping= windows size - 1 sec)
    overlapping = windowSize_sec - 1
    X_rW_IMU_seg = []

    for start in range(0, len(IMU_rW), windowSize_sec*sampleRate_IMU_acc - overlapping*sampleRate_IMU_acc):
        end = start + sampleRate_IMU_acc*windowSize_sec
        if len(IMU_rW[start:end]) < sampleRate_IMU_acc*windowSize_sec:
            break
        else:
            X_rW_IMU_seg.append(IMU_rW[start:end])

    #Preprocessing
    X_rW_IMU_seg_conv = []
    for X_rW_IMU_seg_sam in X_rW_IMU_seg:
        X_rW_IMU_seg_conv.append(X_rW_IMU_seg_sam)

    ##feature extraction
    X_rW_IMU = []
    for X_rW_IMU_seg_sample in X_rW_IMU_seg_conv:
        X_rW_IMU.append(extract_feature_IMU(np.array(X_rW_IMU_seg_sample)))

    ##Normalization
    sc_IMU = joblib.load('./models/sc_IMU.pkl')
    X_rW_IMU_std = sc_IMU.transform(X_rW_IMU)
    
    #Feed into model
    y_IMU_pred = ensemble(X_rW_IMU_std)
    IMU_sample_rw_pred = pred_flatToSamples_IMU(y_IMU_pred)
    
    #Save result
    result = pd.concat([pd.Series(IMU_sample_rw_pred), IMU_rW_pd[['timestamp']][:len(IMU_sample_rw_pred)]], ignore_index=True, axis=1)
    result.to_csv(('C:/Users/NCTU/Desktop/WhoTakeWhat/data/result/Event_Detection/%s.csv' % who), header=None, index=False)

In [6]:
1/0

ZeroDivisionError: division by zero

# Debug

Sign IMU result in Kinect video

In [ ]:
#Sign IMU result in Kinect video
IMU_result_inKinect = []

IMU_rW_time = pd.to_datetime(IMU_rW_pd['timestamp'], format='%H:%M:%S:%f')
Kinect_rW_time = pd.to_datetime(Kinect_rW_pd['timestamp'], format='%H:%M:%S:%f')
i = 0
for Kinect_rW_timestamp in Kinect_rW_time:
    for si in range(i, len(IMU_rW_time)):
        if IMU_rW_time[si] <= Kinect_rW_timestamp and Kinect_rW_timestamp < IMU_rW_time[si+1]:
            IMU_result_inKinect.append(IMU_sample_rw_pred[i])
            i = si
            break
            
# IMU_Kinect_sample_rw_pred = pd.DataFrame(np.array([IMU_result_inKinect, Kinect_sample_rw_pred]).T)

### Save to csv include result of predicting, headX_video, headY_video

In [ ]:
result = pd.concat([pd.Series(IMU_result_inKinect), Kinect_rW_pd[['Head_image_x', 'Head_image_y', 'timestamp']]], ignore_index=True, axis=1)

result.to_csv('D:/GoogleDrive/Graduate/Research/Research_HsinWei/Programs/data/realWorldData/ReachOutResult.csv', header=None, index=False)